# Using OptimalControl Model

In [1]:
using OptimalControl
include("../Problems/OptimalControl/space_Shuttle_OC.jl")
SpaceShuttleOC = space_Shuttle_OC()


The (autonomous) optimal control problem is given by:

    w = 203000.0
    g₀ = 32.174
    m = w / g₀
    ρ₀ = 0.002378
    hᵣ = 23800.0
    Rₑ = 2.09029e7
    μ = 1.4076539e16
    S = 2690.0
    a₀ = -0.20704
    a₁ = 0.029244
    b₀ = 0.07854
    b₁ = -0.0061592
    b₂ = 0.000621408
    c₀ = 1.0672181
    c₁ = -0.019213774
    c₂ = 0.00021286289
    c₃ = -1.0117249e-6
    h_s = 2.6 * 100000.0
    ϕ_s = deg2rad(0)
    θ_s = deg2rad(0)
    v_s = 2.56 * 10000.0
    γ_s = deg2rad(-1)
    ψ_s = deg2rad(90)
    α_s = deg2rad(0)
    β_s = deg2rad(0)
    t_s = 1.0
    h_t = 0.8 * 100000.0
    v_t = 0.25 * 10000.0
    γ_t = deg2rad(-5)
    tf = 2009.0
    t0 = 0.0
    t ∈ [t0, tf], time
    x ∈ R⁶, state
    u ∈ R², control
    h = x₁
    ϕ = x₂
    θ = x₃
    v = x₄
    γ = x₅
    ψ = x₆
    α = u₁
    β = u₂
    h(t) ≥ 0, h_con
    deg2rad(-89) ≤ θ(t) ≤ deg2rad(89), θ_con
    v(t) ≥ 0.0001, v_con
    deg2rad(-89) ≤ γ(t) ≤ deg2rad(89), γ_con
    deg2rad(-89) ≤ β(t) ≤ deg2rad(1), β_con
    


┌────────┬────────┬──────────┬──────────┬───────────┬────────────┬─────────────┐
│ times* │ state* │ control* │ variable │ dynamics* │ objective* │ constraints │
├────────┼────────┼──────────┼──────────┼───────────┼────────────┼─────────────┤
│     ✅ │     ✅ │       ✅ │       ❌ │        ✅ │         ✅ │          ✅ │
└────────┴────────┴──────────┴──────────┴───────────┴────────────┴─────────────┘


In [2]:
import Interpolations
using CTBase
 ## Initial conditions
 h_s = 2.6          # altitude (ft) / 1e5
 ϕ_s = deg2rad(0)   # longitude (rad)
 θ_s = deg2rad(0)   # latitude (rad)
 v_s = 2.56         # velocity (ft/sec) / 1e4
 γ_s = deg2rad(-1)  # flight path angle (rad)
 ψ_s = deg2rad(90)  # azimuth (rad)
 α_s = deg2rad(0)   # angle of attack (rad)
 β_s = deg2rad(0)   # bank angle (rad)
 t_s = 1.00         # time step (sec)

 ## Final conditions, the so-called Terminal Area Energy Management (TAEM)
 h_t = 0.8          # altitude (ft) / 1e5
 v_t = 0.25         # velocity (ft/sec) / 1e4
 γ_t = deg2rad(-5)  # flight path angle (rad)

 ## Number of mesh points (knots) to be used
 n = 503

x_s = [h_s, ϕ_s, θ_s, v_s, γ_s, ψ_s, α_s, β_s]
x_t = [h_t, ϕ_s, θ_s, v_t, γ_t, ψ_s, α_s, β_s]
interp_linear = Interpolations.LinearInterpolation([1, n], [x_s, x_t])
initial_guess = mapreduce(transpose, vcat, interp_linear.(1:n))

xinit = initial_guess[:,1:6]
uinit = initial_guess[:,7:8]

init = CTBase.OCPInit((x_init = xinit,u_init = uinit))


OCPInit(CTBase.var"#237#244"{Nothing}(nothing), CTBase.var"#239#246"{Nothing}(nothing), nothing, #undef, #undef, :constant_or_function)

In [3]:
import HSL_jll
sol = OptimalControl.solve(SpaceShuttleOC,init=init,
                            linear_solver="ma57",hsllib=HSL_jll.libhsl_path)